# Notebook / System Performace Test

Time a simple keras model so can guage relative performance between systems.

In [1]:
import tensorflow as tf
import sys

print(sys.version)
print(tf.__version__)

2022-02-02 16:25:26.715276: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
2.4.1


In [2]:
# preliminary test, see if it looks like we have gpu and cpu devices available to use
# introspect tensorflow to see if gpu devices available or not
# https://www.tensorflow.org/guide/gpu
# https://www.tensorflow.org/api_docs/python/tf/config/list_physical_devices
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

# display all of the device information
for device in tf.config.list_physical_devices():
    print(device)

Num GPUs Available:  1
Num CPUs Available:  1
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2022-02-02 16:25:27.818605: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-02 16:25:27.819381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-02 16:25:27.847918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 16:25:27.848144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro M2200 computeCapability: 5.2
coreClock: 1.036GHz coreCount: 8 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 82.08GiB/s
2022-02-02 16:25:27.848159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-02 16:25:27.849407: I tensorflow/stream_executor/platform/default/dso_loa

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape/flatten tensor for simple dense layer inputs
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

# turn inter labels into a 1-hot encoded categorical output to match our
# model output layer with 10 output units
#train_labels = to_categorical(train_labels)
#test_labels = to_categorical(test_labels)

print(train_images.shape)
print(train_labels.shape)
print(train_labels)

(60000, 784)
(60000,)
[5 0 4 ... 5 6 8]


In [4]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

2022-02-02 16:25:28.153024: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-02 16:25:28.153639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 16:25:28.153881: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro M2200 computeCapability: 5.2
coreClock: 1.036GHz coreCount: 8 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 82.08GiB/s
2022-02-02 16:25:28.153902: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.

In [5]:
# time how long it takes to train and fit model for 10 epochs
with tf.device('/CPU:0'):
    %timeit model.fit(train_images, train_labels, epochs=10, batch_size=128)

2022-02-02 16:25:28.710861: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-02 16:25:28.730719: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.4365 - accuracy: 0.8722
Epoch 2/10
469/469 [==============================] - 2s 3ms/step - loss: 0.1143 - accuracy: 0.9666
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0721 - accuracy: 0.9781
Epoch 4/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0518 - accuracy: 0.9843
Epoch 5/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0378 - accuracy: 0.9895
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0257 - accuracy: 0.9923
Epoch 7/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0204 - accuracy: 0.9944
Epoch 8/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0169 - accuracy: 0.9954
Epoch 9/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0117 - accuracy: 0.9966
Epoch 10/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0089 - accuracy: 0.9978

In [6]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [7]:
# time how long it takes to train and fit model for 10 epochs
with tf.device('/GPU:0'):
    %timeit model.fit(train_images, train_labels, epochs=10, batch_size=128)

Epoch 1/10
 24/469 [>.............................] - ETA: 0s - loss: 1.2783 - accuracy: 0.6040  

2022-02-02 16:28:35.642912: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


469/469 [==============================] - 1s 2ms/step - loss: 0.4286 - accuracy: 0.8743
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1066 - accuracy: 0.9679
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0685 - accuracy: 0.9801
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0468 - accuracy: 0.9856
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0365 - accuracy: 0.9898
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0276 - accuracy: 0.9923
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0200 - accuracy: 0.9942
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0151 - accuracy: 0.9957
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0115 - accuracy: 0.9967
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0094 - accuracy: 0.9976
Epoch 1/10

| System | Result |
|--------|--------|
| Colab default runtime | 1 loop, best of 5: 48.2 s per loop |
| Colab GPU runtime     | 1 loop, best of 5: 17.8 s per loop |
| DevBox on Lenovo P51 (no GPU)    | 31.1 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) |
| Lenovo P51 native (no GPU)       | 15.7 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
| Lenovo P51 native (Quadro M2200) | 8.77 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) |
| TLC gpu02 (no GPU)               | 1min 23s ± 258 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)|
| TLC gpu02 (GeForce GTX 1080 Ti)  | 21.3 s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) |